In [5]:
import numpy as np
import re

In [45]:
# estimate energy consumption using the simple energest output
# Example energest output:
# [INFO: Energest  ] --- Period summary #2 (60 seconds)
# [INFO: Energest  ] Total time  :    1966080
# [INFO: Energest  ] CPU         :      10374/   1966080 (5 permil)
# [INFO: Energest  ] LPM         :    1955706/   1966080 (994 permil)
# [INFO: Energest  ] Deep LPM    :          0/   1966080 (0 permil)
# [INFO: Energest  ] Radio Tx    :        106/   1966080 (0 permil)
# [INFO: Energest  ] Radio Rx    :     104802/   1966080 (53 permil)
# [INFO: Energest  ] Radio total :     104908/   1966080 (53 permil)

# Energest ouput
energest_output = " --- Period summary #2 (60 seconds)\nTotal time  :    1966080\nCPU         :      10374/   1966080 (5 permil)\nLPM         :    1955706/   1966080 (994 permil)\nDeep LPM    :          0/   1966080 (0 permil)\nRadio Tx    :        106/   1966080 (0 permil)\nRadio Rx    :     104802/   1966080 (53 permil)\nRadio total :     104908/   1966080 (53 permil)"

# with regex, we can extract the numbers from the output
# total time
# CPU time
# LPM time
# Radio Tx time
# Radio Rx time
# Radio total time

# regex to extract the numbers
regex = re.compile(r":\s+(\d+)")

# extract the values
match = regex.findall(energest_output)
times = [int(x) for x in match]

total_time, cpu_time, lpm_time, deep_lpm_time, radio_tx_time, radio_rx_time, radio_total_time = times

# print the values
print("Total time: %s" % total_time)
print("CPU time: %s" % cpu_time)
print("LPM time: %s" % lpm_time)
print("Deep LPM: %s" % deep_lpm_time)
print("Radio Tx time: %s" % radio_tx_time)
print("Radio Rx time: %s" % radio_rx_time)
print("Radio total time: %s" % radio_total_time)

# current draw in different states in mA
# CPU_CURRENT = 10
# LPM_CURRENT = 0.023
# DEEP_LPM_CURRENT = 0
# RADIO_TX_CURRENT = 18.4
# RADIO_RX_CURRENT = 17.4

CURRENT = {
    "CPU": 10,
    "LPM": 0.023,
    "DEEP_LPM": 0,
    "RADIO_TX": 18.4,
    "RADIO_RX": 17.4
}

# 3 volt battery
VOLTAGE = 3

# we have the following values available
# []_time - the number of ticks spent in each mode
# TICKS_PER_SECOND - the number of ticks per second
# total_time - the total time in ticks
# []_CURRENT - the current draw in each mode
# voltage - the provided voltage to the component

# state_avg_current_mA = ([]_time * current_mA) / total_time
# state_charge_mC = ([]_time * current_mA) / TICKS_PER_SECOND
# state_charge_mA = state_charge_mC / 3600
# state_power_mW = avg_current_mA * voltage
# state_energy_mJ = state_charge_mC * voltage

TICKS_PER_SECOND = 32768

# calculate the average current in each state
values = [
    {
        f"{state}_avg_current_mA": (state_time * state_current) / total_time,
        f"{state}_charge_mC": (state_time * state_current) / TICKS_PER_SECOND,
        f"{state}_charge_mA": (state_time * state_current) / TICKS_PER_SECOND / 3600,
        f"{state}_power_mW": (state_time * state_current) / total_time * VOLTAGE,
        f"{state}_energy_mJ": (state_time * state_current) / TICKS_PER_SECOND * VOLTAGE
    } for state_time, state_current, state in zip(times[:-1], CURRENT.values(), CURRENT.keys())
]

for v in values:
    print(v)


Total time: 1966080
CPU time: 10374
LPM time: 1955706
Deep LPM: 0
Radio Tx time: 106
Radio Rx time: 104802
Radio total time: 104908
{'CPU_avg_current_mA': 10.0, 'CPU_charge_mC': 600.0, 'CPU_charge_mA': 0.16666666666666666, 'CPU_power_mW': 30.0, 'CPU_energy_mJ': 1800.0}
{'LPM_avg_current_mA': 0.0001213592529296875, 'LPM_charge_mC': 0.00728155517578125, 'LPM_charge_mA': 2.0226542154947915e-06, 'LPM_power_mW': 0.0003640777587890625, 'LPM_energy_mJ': 0.02184466552734375}
{'DEEP_LPM_avg_current_mA': 0.0, 'DEEP_LPM_charge_mC': 0.0, 'DEEP_LPM_charge_mA': 0.0, 'DEEP_LPM_power_mW': 0.0, 'DEEP_LPM_energy_mJ': 0.0}
{'RADIO_TX_avg_current_mA': 0.0, 'RADIO_TX_charge_mC': 0.0, 'RADIO_TX_charge_mA': 0.0, 'RADIO_TX_power_mW': 0.0, 'RADIO_TX_energy_mJ': 0.0}
{'RADIO_RX_avg_current_mA': 0.0009381103515625, 'RADIO_RX_charge_mC': 0.056286621093749996, 'RADIO_RX_charge_mA': 1.5635172526041666e-05, 'RADIO_RX_power_mW': 0.0028143310546875, 'RADIO_RX_energy_mJ': 0.16885986328125}


In [43]:
state_time, state_current, state = list(zip(times[:-1], CURRENT.values(), CURRENT.keys()))[0]

1966080 10 CPU
